In [1]:

####################################################################################################
####################################################################################################

import sys
import os

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint

from IPython.display import clear_output
from IPython.display import display

sys.path.append(os.path.abspath("../"))
from pax_utils import event_utils
from pax_utils import file_utils
from pax_utils import interaction_utils
from pax_utils import s1s2_utils
from pax_utils import numeric_utils
from pax_utils import waveform_pax_utils
from pax_utils import waveform_utils

sys.path.append(os.path.abspath("../pax"))
from pax import core



In [4]:

####################################################################################################
####################################################################################################

f = plt.figure(figsize=(20, 10))

def processPklEvents(zipfilename, iZip, dir_waveforms_s2):

    ################################################################################################
    ################################################################################################
    
    lst_pkl_files  = file_utils.getListOfPklFilesFromZipFile(zipfilename, nEventsPerFile)

    
    ################################################################################################
    ################################################################################################
    
    sum_wf = None
    #event  = None
    df_zip_merged   = pd.DataFrame()
    df_s2_waveforms = pd.DataFrame()
        
    ################################################################################################
    ################################################################################################
    
    for iPklFile in range(0, len(lst_pkl_files)):
    
      
        ############################################################################################
        ############################################################################################

        event_number = iZip*1000 + iPklFile
        
        file_out_s2_waveforms = dir_waveforms_s2 + '/' + 'event' + format(event_number, '04d') + '_S2waveforms' + '.pkl'
        
        
        ############################################################################################
        ############################################################################################
        
        pklfilename  = lst_pkl_files[iPklFile]
        event        = file_utils.getPaxEventFromPklFileInZipArchive(zipfilename, pklfilename)
 

        ############################################################################################
        ############################################################################################
        
        print("   PKL File: '" + str(pklfilename) + "'")
        #print("   event_number: " + str(iPklFile))
        #clear_output(wait=True)
        
        
        ############################################################################################
        ############################################################################################
        
        interactions  = event.interactions
        nInteractions = len(interactions)
        
        #if (nInteractions != 1): continue
            
            
        ############################################################################################
        ############################################################################################
        
        df_pkl_event  = event_utils.getEventDataFrameFromEvent(event)
        df_pkl_intr   = interaction_utils.getInteractionDataFrameFromEvent(event)
        df_pkl_s2s    = s1s2_utils.getS2integralsDataFrame(event, 127)
        
                
        ############################################################################################
        ############################################################################################
        
        left                        = event.main_s2.left
        right                       = event.main_s2.right
        jsonfilename                = os.path.dirname(zipfilename) + '/pax_info.json'
        cfg                         = event_utils.getConfig(jsonfilename)

        
        ############################################################################################
        # Get summed S2 waveform from event, PAX
        ############################################################################################
        
        arr_summed_waveform_top_evt = waveform_pax_utils.getSummedWaveformFromEvent(event, 'tpc_top')
        arr_summed_waveform_top_evt = arr_summed_waveform_top_evt[left:right]
        
        
        ############################################################################################
        # Get summed S2 waveform from PAX
        ############################################################################################
        
        #arr_summed_waveform_top_pax = waveform_pax_utils.SumWaveformPAX(event)
        #arr_summed_waveform_top_pax = arr_summed_waveform_top_evt[left:right]
                
            
        ############################################################################################
        # Get dataframe of S2 waveform for each PMT channel
        ############################################################################################
        
        df_channels_waveforms_top     = waveform_utils.getChannelsWaveformsDataFrame(event, cfg, 'top', False)
        df_channels_waveforms_top_all = waveform_utils.addEmptyChannelsToDataFrame(df_channels_waveforms_top).copy(deep=True)
        
        
        ############################################################################################
        # Get summed S2 waveform from dataframe
        ############################################################################################
        
        arr_summed_waveform_top_df  = waveform_utils.getSummedWaveformFromDataFrame(df_channels_waveforms_top_all, event)
        arr_summed_waveform_top_df  = arr_summed_waveform_top_df[left:right]
        
        
        ############################################################################################
        # Check that the S2 summed waveform from the event and dataframe are equal
        ############################################################################################
        
        wf_arrs_equal = numeric_utils.compareArrays(arr_summed_waveform_top_evt, arr_summed_waveform_top_df)
        
        assert(wf_arrs_equal)
        
        
        ############################################################################################
        # Check that the integrals of the S2 summed waveform from the event and dataframe are equal
        ############################################################################################
        
        #sum_summed_waveform_top_pax    = np.sum(arr_summed_waveform_top_pax)
        sum_summed_waveform_top_evt     = np.sum(arr_summed_waveform_top_evt)
        sum_summed_waveform_top_df      = np.sum(arr_summed_waveform_top_df)
        
        sum_summed_waveforms_equal = numeric_utils.compareFloats(sum_summed_waveform_top_evt, sum_summed_waveform_top_df, 1e-4)
               
        assert(sum_summed_waveforms_equal)
    
        
        ############################################################################################
        # Check that the per-channel S2 integrals from the event and dataframe are equal
        ############################################################################################

        arr_s2integrals_evt = df_pkl_s2s.iloc[0][1:].as_matrix().astype(np.float32)
        arr_s2integrals_df  = df_channels_waveforms_top_all[:]['sum'].as_matrix().astype(np.float32)
        
        arr_s2integrals_equal = numeric_utils.compareArrays(arr_s2integrals_evt, arr_s2integrals_df)
        
        assert(arr_s2integrals_equal)

        
        ############################################################################################
        # Check that the sum of the per-channel S2 integrals from the event and dataframe are equal
        ############################################################################################
        
        sum_s2integrals_evt = np.sum(arr_s2integrals_evt)
        sum_s2integrals_df  = np.sum(arr_s2integrals_df)
        

        # this fails use weaker condition for now
        #sum_s2integrals_equal = numeric_utils.compareFloats(sum_s2integrals_evt, sum_s2integrals_df)
        
        assert( abs(sum_s2integrals_evt - sum_s2integrals_df) / sum_s2integrals_df < 0.01) # this fails
        
        
        ############################################################################################
        ############################################################################################
        
        print()
        print("Integral of Summed Waveform Event:       " + str(sum_summed_waveform_top_evt))
        print("Integral of Summed Waveform DF:          " + str(sum_summed_waveform_top_df))
        print("Sum of S2 Integrals over channels Event: " + str(sum_s2integrals_evt))
        print("Sum of S2 Integrals over channels DF:    " + str(sum_s2integrals_df))
        print()
        
        
        
        
        ############################################################################################
        ############################################################################################
    
        df_pkl_merged = df_pkl_event.merge(df_pkl_intr).merge(df_pkl_s2s)
        df_zip_merged = df_zip_merged.append(df_pkl_merged)
        
        
        ############################################################################################
        # Save S2 Waveforms
        ############################################################################################

        display(df_channels_waveforms_top[0:5][:])
        
        df_channels_waveforms_top.to_pickle(file_out_s2_waveforms)
        
        
        ############################################################################################
        ############################################################################################
        
        continue
      
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged.reset_index(inplace=True, drop=True)
       
    return df_zip_merged, df_s2_waveforms

    

In [5]:
import glob

####################################################################################################
# input
####################################################################################################

nEventsPerFile = 2 #1000
nFilesZip      = 2 #200
if (nFilesZip == -1): nFilesZip = len(lst_contents)
nEvents        = nEventsPerFile * nFilesZip

dir_input    = '../pax_run/pax_output/Mar25/'
dir_format   = dir_input + "instructions_" + ('[0-9]' * 6)
file_format  = 'XENON1T-0-000000000-000000999-000001000.zip'
lst_contents = glob.glob(dir_format)


####################################################################################################
# output
####################################################################################################

ver              = datetime.datetime.now().strftime("%y%m%d%H%M")
ver              = 'test'
dir_out_pkl      = 'merged' + '/'
file_pkl         = dir_out_pkl + 'merged_pax_' + str(nEvents % 1000) + 'k_' + ver + '.pkl'
dir_waveforms    = dir_out_pkl + '/' + 'waveforms_' + ver
dir_waveforms_s2 = dir_waveforms + '/' + 's2'

if(not os.path.isdir(dir_waveforms)):
    os.mkdir(dir_waveforms)
    if(not os.path.isdir(dir_waveforms_s2)):
        os.mkdir(dir_waveforms_s2)
    else:
        print("\nDirectory '" + dir_waveforms_s2 + "' exists!\n") 
else:
    print("\nDirectory '" + dir_waveforms + "' exists!\n")


#print()
#print("PAX input files: " + str(nFilesZip))
#print()


####################################################################################################
####################################################################################################

df = pd.DataFrame()

for iZip in range(0, nFilesZip):
    
    ################################################################################################
    ################################################################################################
    
    zipfilename = lst_contents[iZip] + '/' + file_format
    
    if (not os.path.exists(zipfilename)):
        
        print("Error! File: '" + str(zipfilename) + "' does not exist.")
    
        continue
        
    print("Zip File: '" + str(zipfilename) + "'")
    
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged, df_s2_waveforms = processPklEvents(zipfilename, iZip, dir_waveforms_s2)

    
    ################################################################################################
    ################################################################################################
    
    df = df.append(df_zip_merged)
    
    continue
    

print("Done")


####################################################################################################
# Write
####################################################################################################

display(df[0:1][:])
print(file_pkl)

df.to_pickle(file_pkl)




Directory 'merged//waveforms_test' exists!

Zip File: '../pax_run/pax_output/Mar25/instructions_000000/XENON1T-0-000000000-000000999-000001000.zip'
   PKL File: '0'

Integral of Summed Waveform Event:       260.568
Integral of Summed Waveform DF:          260.568
Sum of S2 Integrals over channels Event: 259.256
Sum of S2 Integrals over channels DF:    260.568



,channel,left,right,length,sum,raw_data
0,0,82932,83327,396,1.435604,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,3,82836,83005,170,2.190272,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,4,82886,83013,128,3.737455,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,5,81914,83453,1540,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,9,82904,97801,14898,4.321683,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


   PKL File: '1'

Integral of Summed Waveform Event:       699.917
Integral of Summed Waveform DF:          699.917
Sum of S2 Integrals over channels Event: 699.917
Sum of S2 Integrals over channels DF:    699.917



,channel,left,right,length,sum,raw_data
0,0,105424,105791,368,1.715228,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,4,65458,148393,82936,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,5,10394,127133,116740,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,9,105400,105501,102,1.293108,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,10,105580,105681,102,1.385255,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Zip File: '../pax_run/pax_output/Mar25/instructions_000001/XENON1T-0-000000000-000000999-000001000.zip'
   PKL File: '0'

Integral of Summed Waveform Event:       483.124
Integral of Summed Waveform DF:          483.124
Sum of S2 Integrals over channels Event: 483.126
Sum of S2 Integrals over channels DF:    483.124



...

   PKL File: '1'

Integral of Summed Waveform Event:       243.206
Integral of Summed Waveform DF:          243.206
Sum of S2 Integrals over channels Event: 243.202
Sum of S2 Integrals over channels DF:    243.206



,channel,left,right,length,sum,raw_data
0,5,57434,116573,59140,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,6,120362,120463,102,1.794950,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7,120476,120575,100,0.904435,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,8,120602,120703,102,0.933865,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,15,19424,120451,101028,1.188072,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Done


,event_number,event_n_pulses,event_pulses_count,event_s2_count,event_s2_left,event_s2_right,event_s2_length,intr_count,intr_x,intr_y,...,s2_area_117,s2_area_118,s2_area_119,s2_area_120,s2_area_121,s2_area_122,s2_area_123,s2_area_124,s2_area_125,s2_area_126
0,0,190,190,3,82836,83317,482,0,-2.618421,15.835213,...,1.584757,0.0,2.464882,0.924652,0.0,1.981129,3.65679,0.0,1.322134,2.032758


merged/merged_pax_4k_test.pkl
